# Exploratory data analysis on the San Francisco 311 data for 1 Hour time intervals
Data can be downloaded from: https://data.sfgov.org/City-Infrastructure/311-Cases/vw6y-z8j6/data

In [ ]:
%matplotlib inline
%load_ext autoreload
import matplotlib.pyplot as plt

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np

## The time window to bucket samples
TIME_RANGE = '1H'

## File path (original data is ~1GB, this is a reduced version with only categories and dates)
#Original file:
#DATAPATH = "SF311_simplified.csv"

#Sample raw data:
DATAPATH = "SF_data/SF-311_simplified.csv"

### Read sample of data (original data contains additional columns)

In [ ]:
raw_sample = pd.read_csv(DATAPATH, nrows=5)
raw_sample.head()

In [ ]:
raw = pd.read_csv(DATAPATH).drop(columns='Unnamed: 0')
raw.head(10)

#### Initial data prep

In [ ]:
## Rename columns
raw = raw.rename(columns={'Opened': 'date', 'Category': 'category'})

## Turn the raw data into a time series (with date as DatetimeIndex)
from moda.dataprep.raw_to_ts import raw_to_ts
ts = raw_to_ts(raw,date_format="%m/%d/%Y %H:%M:%S %p")

In [ ]:
ts.head()

In [ ]:
## Some general stats

print("Dataset length: " + str(len(ts)))
print("Min date: " + str(ts.index.get_level_values('date').min()))
print("Max date: " + str(ts.index.get_level_values('date').max()))

print("Total time: {}".format(ts.index.get_level_values('date').max() - ts.index.get_level_values('date').min()))

print("Dataset contains {} categories.".format(len(ts['category'].unique())))



#### Next, we decide on the time interval and aggregate items per time and category

In [ ]:
from moda.dataprep.ts_to_range import ts_to_range
ranged_ts = ts_to_range(ts,time_range=TIME_RANGE)
ranged_ts.head(20)

In [ ]:
#I'm using dfply because I like its functional-like syntax. This can also be done with plain pandas.
#!pip install dfply
from dfply import *

## Remove categories with less than 1000 items (in more than 10 years) or that existed less than 100 days
min_values = 1000
min_days = 100



categories = ranged_ts.reset_index() >> group_by(X.category) >> \
    summarise(value = np.sum(X.value),duration_in_dataset = X.date.max()-X.date.min()) >> \
    ungroup() >> \
    mask(X.duration_in_dataset.dt.days > min_days) >> \
    mask(X.value > min_values) >> \
    arrange(X.value,ascending=False)



print("Filtered dataset contains {0} categories,\nafter filtering the small ones that existed less than {1} days or had {2} values of less.".
      format(len(categories),min_days,min_values))

categories.head()

### Most common categories

In [ ]:
category_names = categories['category'].values
num_categories = len(categories)

major_category_threshold=11
major_categories = category_names[:major_category_threshold]
minor_categories = category_names[major_category_threshold:]

fig, axes = plt.subplots(nrows=1, ncols=2)
fig.set_figheight(5)
fig.set_figwidth(20)

categories[categories['category'].isin(major_categories)].plot(kind='bar',
                                                               x='category',
                                                               y='value',
                                                               title="Top "+str(major_category_threshold-1)+" common categories on the SF311 dataset",
                                                               ax=axes[0])
categories[categories['category'].isin(minor_categories)].plot(kind='bar',
                                                               x='category',
                                                               y='value',
                                                               title=str(major_category_threshold)+"th to "+str(num_categories)+"th most common categories on the SF311 dataset",
                                                               ax=axes[1])

plt.savefig("category_values.png",bbox_inches='tight')

### Change in requests per category from year to year

In [ ]:
## Calculate the number of values per category per year
categories_yearly = ranged_ts.reset_index() >> mutate(year = X.date.dt.year) >> group_by(X.category,X.year) >> \
    summarise(value_per_year = np.sum(X.value),
              duration_in_dataset = X.date.max()-X.date.min()) >>\
    ungroup() >> \
    mask(X.value_per_year > (min_values/12.0)) >> \
    arrange(X.value_per_year,ascending=False)

import seaborn as sns

major_cats_yearly = categories_yearly[categories_yearly['category'].isin(major_categories)]

g = sns.factorplot(x='category', y='value_per_year', hue='year', data=major_cats_yearly, kind='bar', size=4, aspect=4,legend=True)
g.set_xticklabels(rotation=90)
axes = g.axes.flatten()
axes[0].set_title("Yearly number of incidents for the top "+str(major_category_threshold-1)+" categories")
plt.savefig("yearly_values.png",bbox_inches='tight')

In [ ]:
minor_cats_yearly = categories_yearly[categories_yearly['category'].isin(minor_categories)]

g = sns.factorplot(x='category', y='value_per_year', hue='year', data=minor_cats_yearly, kind='bar', size=4, aspect=4,legend=True)
g.set_xticklabels(rotation=90)
axes = g.axes.flatten()
axes[0].set_title("Yearly number of incidents for the "+str(major_category_threshold)+"th to "+str(num_categories)+"th categories")

### Correlation between categories over time

In [ ]:
categories_yearly_pivot = categories_yearly.pivot("year", "category", "value_per_year")
categories_yearly_pivot.head()
corr = categories_yearly_pivot.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

### One category inspection
#### Example 1: Noise Reports

In [ ]:
category = "Noise Report"
ranged_ts.loc[pd.IndexSlice[:, category], :].reset_index().plot(kind='line',x='date',y='value',figsize=(24,6),linewidth=0.7, 
                          title = "Number of incidents per 1 hour for {}".format(category))

#### Example 2: Street and Sidewalk Cleaning

In [ ]:
START = '2015-11-01'
END = '2018-07-01'
category = 'Street and Sidewalk Cleaning'
cleaning = ranged_ts.loc[pd.IndexSlice[:, category], :].reset_index()
cleaning[(cleaning.date > START) & (cleaning.date<=END)].plot(kind='line',x='date',y='value',figsize=(24,6),linewidth=0.7, 
                          title = "Number of incidents per 1 hours for {0} between {1} and {2}".format(category,START,END))

As comparison, let's look at the same time series with different time ranges (30 minutes, 1 hour and 24 hours), only on two months of data

In [ ]:
from moda.dataprep.ts_to_range import ts_to_range
ranged_ts_3H = ts_to_range(ts,time_range='3H',pad_with_zeros=True)
ranged_ts_30min = ts_to_range(ts,time_range='30min',pad_with_zeros=True)

START = '2015-11-01'
END = '2016-01-01'
category = 'Street and Sidewalk Cleaning'

fig, axes = plt.subplots(nrows=3, ncols=1,figsize=(20,12))

cleaning_30min = ranged_ts_30min.loc[pd.IndexSlice[:, category], :].reset_index()
a1=cleaning_30min[(cleaning_30min.date > START) & (cleaning_30min.date<=END)].plot(kind='line',x='date',y='value',linewidth=0.7, ax=axes[0])

cleaning_3H = ranged_ts_3H.loc[pd.IndexSlice[:, category], :].reset_index()
a2=cleaning_3H[(cleaning_3H.date > START) & (cleaning_3H.date<=END)].plot(kind='line',x='date',y='value',linewidth=0.7, ax=axes[1])

cleaning_1H = ranged_ts.loc[pd.IndexSlice[:, category], :].reset_index()
a3=cleaning_1H[(cleaning_1H.date > START) & (cleaning_1H.date<=END)].plot(kind='line',x='date',y='value',linewidth=0.7, ax=axes[2])


We can see that there are multiple seasonality factors in this time series. Hourly and weekly patterns are visible on the 30 minute interval time series, and the 3 hours interval time series

## Evaluating different models on the SF 1H data

First, in order to be able to estimate our models, we use [TagAnomaly](https://github.com/Microsoft/TagAnomaly) to tag the points we think are showing trends in the data. Taganomaly can be found here: https://github.com/Microsoft/TagAnomaly
Second, we join the tagged dataset with the time series dataset. Each sample which isn't included in the tagged dataset is assumed to be non-trending (or normal)

In [ ]:
## Add labeled data
labels1H = pd.read_csv('SF_data/SF_1H_anomalies_only.csv',usecols=['date','category','value'])
labels1H.date = pd.to_datetime(labels1H.date)

labels1H['label'] = 1
labels1H.sort_values(by='date').head()

In [ ]:
# Since we have labels only for 2018, we'll filter out previous years.
ts2018 = ranged_ts[ranged_ts.index.get_level_values(0).year == 2018]
ts2018.head()

In [ ]:
df1H = pd.merge(ts2018.reset_index(),labels1H,how='left',on=['date','category'])
df1H['label'] = np.where(np.isnan(df1H['value_y']),0,1)
df1H = df1H.set_index([pd.DatetimeIndex(df1H['date']),'category'])
df1H = df1H.drop(columns = ['date','value_y']).rename(columns = {'value_x':'value'})
df1H.head()

In [ ]:
from moda.evaluators import get_metrics_for_all_categories, get_final_metrics
from moda.dataprep import read_data
from moda.models import TwitterAnomalyTrendinessDetector, MovingAverageSeasonalTrendinessDetector, \
    STLTrendinessDetector, AzureAnomalyTrendinessDetector


def run_model(dataset, freq, min_date='01-01-2018', plot=False, model_name='stl', min_value=10,
              min_samples_for_category=100):


    if model_name == 'twitter':
        model = TwitterAnomalyTrendinessDetector(is_multicategory=True, freq=freq, min_value=min_value, threshold=None,
                                                 max_anoms=0.49, seasonality_freq=7)

    if model_name == 'ma_seasonal':
        model = MovingAverageSeasonalTrendinessDetector(is_multicategory=True, freq=freq, min_value=min_value,
                                                        anomaly_type='or',
                                                        num_of_std=3)

    if model_name == 'stl':
        model = STLTrendinessDetector(is_multicategory=True, freq=freq, min_value=min_value,
                                      anomaly_type='residual',
                                      num_of_std=3, lo_delta=0)

    if model_name == 'azure':
        dirname = os.path.dirname(__file__)
        filename = os.path.join(dirname, 'config/config.json')
        subscription_key = get_azure_subscription_key(filename)
        model = AzureAnomalyTrendinessDetector(is_multicategory=True, freq=freq, min_value=min_value,
                                               subscription_key=subscription_key)
    
    # There is no fit/predict here. We take the entire time series and can evaluate anomalies on all of it or just the last window(s)
    prediction = model.predict(dataset, verbose=False)
    raw_metrics = get_metrics_for_all_categories(dataset[['value']], prediction[['prediction']], dataset[['label']],
                                                 window_size_for_metrics=5)
    metrics = get_final_metrics(raw_metrics)
    print(metrics)

    ## Plot each category
    if plot:
        print("Plotting...")
        model.plot(labels=dataset['label'],savefig=False)

    return prediction


In [ ]:
prediction_stl = run_model(df1H,freq='1H',model_name='stl')

In [ ]:
def plot_one_category(category_dataset,model_name='stl'):

    def ts_subplot(plt, series, label):
        plt.plot(series, label=label, linewidth=0.5)
        plt.legend(loc='best')
        plt.xticks(rotation=90)

    plt.subplot(421, )
    ts_subplot(plt, category_dataset['value'], label='Original')
    if 'residual_anomaly' in category_dataset:
        plt.subplot(422)
        ts_subplot(plt, category_dataset['residual_anomaly'], label='Residual anomaly')
    if 'trend' in category_dataset:
        plt.subplot(423)
        ts_subplot(plt, category_dataset['trend'], label='Trend')
    if 'trend_anomaly' in category_dataset:
        plt.subplot(424)
        ts_subplot(plt, category_dataset['trend_anomaly'], label='Trend anomaly')
    if 'seasonality' in category_dataset:
        plt.subplot(425)
        ts_subplot(plt, category_dataset['seasonality'], label='Seasonality')
    
    plt.subplot(426)
    ts_subplot(plt, category_dataset['prediction'], label='Prediction')
    
    if 'residual' in category_dataset:
        plt.subplot(427)
        ts_subplot(plt, category_dataset['residual'], label='Residual')

    plt.subplot(428)
    ts_subplot(plt, category_dataset['label'], label='Labels')
    
    category = category_dataset.category[0]
    
    plt.suptitle("{} results for category {}".format(model_name, category))



In [ ]:
graffiti = prediction_stl.loc[pd.IndexSlice[:, 'Graffiti'], :].reset_index(level='category', drop=False)
fig = plt.figure(figsize=(20,8))
plot_one_category(graffiti,model_name='STL')

The time series in this case is relatively noisy. The model was more conservative than the labeler in this case.

In [ ]:
sewer = prediction_stl.loc[pd.IndexSlice[:, 'Sewer Issues'], :].reset_index(level='category', drop=False)
fig = plt.figure(figsize=(20,8))
plot_one_category(sewer,model_name='STL')

In this case, we missed the first peak as we didn't have enough historical data to estimate it. Let's compare this result to a different model:

In [ ]:
prediction_ma = run_model(df1H,freq=TIME_RANGE,model_name='ma_seasonal')

In [ ]:
sewer2 = prediction_ma.loc[pd.IndexSlice[:, 'Sewer Issues'], :].reset_index(level='category', drop=False)
fig = plt.figure(figsize=(20,8))
plot_one_category(sewer2,model_name='MA seasonal')

This model estimates the trend differently, and found some anomalies on the trend series as well. It too couldn't detect the first peak as it requires some historical data to estimate standard deviation and other statistics.